# Invoke Video Analysis Task - video clip based

You can create a video analysis task using the tool’s UI or by calling the underlying API directly. This notebook provides an example of how to prepare the required metadata and invoke the task through the Lambda function.

In [ ]:
import boto3
import json
import uuid

In [ ]:
lambda_client = boto3.client("lambda")
s3 = boto3.client("s3")

In [ ]:
session = boto3.session.Session()
sts = session.client("sts")
# Get current account ID
account_id = sts.get_caller_identity()["Account"]
# Get current region
region = session.region_name

In [ ]:
S3_BUCKET = f"bedrock-mm-{account_id}-{region}"
S3_KEY_TASK_TEMPLATE = "tasks/{task_id}/upload/{file_name}"

LAMBDA_FUN_NAME_START_TASK = "bedrock-mm-extr-srv-api-start-task"
LAMBDA_FUN_NAME_DELETE_TASK = "bedrock-mm-extr-srv-api-delete-task"

## Invoke clip based video task

The following parameters are required for the frame-sample-based video analysis workflow:
- local_path: the location on your local disk where your video is stored for analysis
- task_id: A unique identifier for the video task
- s3_key: The S3 object key where the video will be stored
- task_name: The name of the task as it will appear in the tool UI (defaults to the video file name if not provided)
- requested_by: The user who submitted the request, for tracking purposes

In [ ]:
local_path = "./samples/content-moderation-demo.mp4"

# generate task_id
task_id = str(uuid.uuid4())

# Construct target s3 key
file_name = local_path.split('/')[-1]
s3_key = S3_KEY_TASK_TEMPLATE.format(task_id=task_id, file_name=file_name)

# The other values required by the request
task_name = file_name # Set your task name, default using the file name
request_by = "workshop-user" # Set your request user name

Upload the video from local disk to S3

In [ ]:
s3.upload_file(local_path, S3_BUCKET, s3_key)

The following JSON sample serves as a template, showing the detailed parameters required for a frame-sample-based task. You can also manage this through the tool’s UI.

In [ ]:
request = {
      "TaskId": task_id,
      "FileName": file_name,
      "TaskName": task_name,
      "Video": {
        "S3Object": {
          "Bucket": S3_BUCKET,
          "Key": s3_key
        }
      },
      "TaskType": "clip",
      "RequestBy": request_by,
      "PreProcessSetting": {
        "StartSec": None,
        "LengthSec": None,
        "UseFixedLengthSec": None,
        "MinClipSec": None, #Default: 4, Min: 1, Max: 5.
      },
      "ExtractionSetting": {
        "Audio": {
          "Transcription": True
        },
        "Vision": {
          "Shot": {
            "Embedding": {
              "Enabled": True,
              "ModelId": "amazon.nova-2-multimodal-embeddings-v1:0",
              "Dimension": 1024
            },
            "Understanding": {
              "Enabled": True,
              "PromptConfigs": [
                {
                  "name": "Content Moderation",
                  "modelId": "amazon.nova-lite-v1:0",
                  "prompt": "You are a content moderation expert tasked with reviewing videos to detect unsafe content",
                  "toolConfig": {
                    "tools": [
                      {
                        "toolSpec": {
                          "name": "tool_result",
                          "description": "Detect whether the provided video contains any unsafe content and classify it into the following categories.",
                          "inputSchema": {
                            "json": {
                              "type": "object",
                              "properties": {
                                "suggestive": {
                                  "type": "boolean",
                                  "description": "Indicates if the video contains bare-chested males, revealing clothing, or other inappropriate scenes."
                                },
                                "tobacco": {
                                  "type": "boolean",
                                  "description": "Indicates if the video contains any scenes including but not limited to bare-chested males, revealing clothing, or other sexually suggestive content.."
                                },
                                "alcohol": {
                                  "type": "boolean",
                                  "description": "Indicates if the video contains any scenes involving alcohol."
                                }
                              },
                              "required": [
                                "suggestive",
                                "tobacco",
                                "alcohol"
                              ]
                            }
                          }
                        }
                      }
                    ]
                  },
                  "inferConfig": {
                    "maxTokens": 500,
                    "topP": 0.1,
                    "temperature": 0.7
                  }
                }
              ]
            }
          }
        }
      },

    }



Invoke the task by calling the Lambda function

In [ ]:
response = lambda_client.invoke(
    FunctionName=LAMBDA_FUN_NAME_START_TASK,
    InvocationType="RequestResponse",  # async
    Payload=json.dumps(request),
)

# Parse the Lambda response
response_payload = response["Payload"].read()
response_payload

## Cleanup
We will call the delete-task endpoint, which removes all task-related state, including the uploaded video (in S3), all metadata files generated in S3, and the metadata stored in DynamoDB. For MME tasks, it will also delete the video-related vectors from the S3 vector index.

In [ ]:
response = lambda_client.invoke(
    FunctionName=LAMBDA_FUN_NAME_DELETE_TASK,
    InvocationType="RequestResponse",  # async
    Payload=json.dumps({"TaskId": task_id}),
)

# Parse the Lambda response
response_payload = response["Payload"].read()
response_payload